In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

import matplotlib.pyplot as plt
import os
import pathlib
import pandas as pd

%matplotlib inline
AUTOTUNE = tf.data.experimental.AUTOTUNE

import sys
sys.path.append('..')
from utilities import Timer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, InputLayer, Dropout, Dense
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy

# Load data as a TensorFlow dataset

Largely based on the TensorFlow [tutorial](https://www.tensorflow.org/tutorials/load_data/images)

In [ ]:
width = height = 224
batch_size = 128

In [ ]:
# Load the classnames

df = pd.read_excel('../dataset/artist-breakdown-annotated.xlsx')
df = df[df.fillna(0).keep.astype(bool)]
class_names = df.artist.unique()
class_names.sort()

In [ ]:
def load_dataset(data_subdir, 
                 shuffle = True, shuffle_buffer_size=1000,
                 batch_size = batch_size,
                 width      = width,
                 height     = height
                ):
    # Load the list of file names in the form of data_dir/class_name/file_name.jpg
    data_dir = pathlib.Path('../dataset/images/') / pathlib.Path(data_subdir)
    list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

    dataset_size = len(list(list_ds))
    
    # Function to get the label
    def get_label(file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        return parts[-2] == class_names
    
    # Function to get the image
    def decode_img(file_path):
        # Load the image
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        
        # Scale image pixels to 0/1
        img = tf.image.convert_image_dtype(img, tf.float32)
        # Resize
        return tf.image.resize(img, [width, height])
    
    # Combine both processes
    def process_path(file_path):
        return decode_img(file_path), get_label(file_path)
    
    # Create the dataset
    ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    ds = ds.cache()
    
    if shuffle:
        if shuffle_buffer_size is not None:
            ds = ds.shuffle(buffer_size = shuffle_buffer_size)
        else:
            ds = ds.shuffle(buffer_size = dataset_size)
    
    ds = ds.repeat().batch(batch_size).prefetch(buffer_size = AUTOTUNE)    
    return ds, dataset_size

In [ ]:
# Look at a batch
peek, peek_size = load_dataset('train', shuffle_buffer_size=32)
image_batch, label_batch = next(iter(peek))

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(class_names[label_batch[n]==1][0].title())
        plt.axis('off')

show_batch(image_batch.numpy(), label_batch.numpy())

## Build Classifier

The original DeepDream was built using [GoogLeNet](https://arxiv.org/abs/1409.4842) which won the ILSVRC (ImageNet Large Scale Visual Recognition Competition) in [2014](http://image-net.org/challenges/LSVRC/2014/).

In the spirit of keeping things simple, a first attempt will be made using a VGGNet (from the [Visual Geometry Group at Oxford](https://www.robots.ox.ac.uk/~vgg/)). This placed 2nd in ILSVRC 2014 but is substantially simpler in architecture, being essentially a vanilla convolutional net.

We don't want to use the pre-existing weights, so we will attempt to train from scratch. You can see the model on the tensorflow [github](https://github.com/tensorflow/tensorflow/blob/23c3bdaacdc27bb82dfd1772efefad687508923a/tensorflow/python/keras/applications/vgg19.py). Note that it is not precisely the same as described in the paper

In [ ]:
model = VGG19(include_top = True, weights = None, input_shape = (width, height, 3), classes = len(class_names))
model.summary()

In [ ]:
optimizer = Adam()
metrics   = [
    CategoricalAccuracy(name = 'accuracy'),
    TopKCategoricalAccuracy(3, name = 'top-3-accuracy'),
    TopKCategoricalAccuracy(5, name = 'top-5-accuracy'),
]

model.compile(optimizer, 
              loss = CategoricalCrossentropy(),
              metrics = metrics)

In [ ]:
train, train_size       = load_dataset('train')
validate, validate_size = load_dataset('validate', shuffle=False)

In [ ]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, TerminateOnNaN

In [ ]:
callbacks = [
    CSVLogger('./logs/training-log.csv', append = True),
    EarlyStopping(patience = 5, restore_best_weights = True),
    ModelCheckpoint(filepath='vgg19-{epoch:02d}-{val_loss:.2f}.hdf5'),
    TerminateOnNaN()
]

In [ ]:
history = model.fit(
    train,
    validation_data  = validate,
    epochs           = 1,
    verbose          = 1,
    callbacks        = callbacks,
    validation_steps = validate_size // batch_size + 1,
    steps_per_epoch  = train_size    // batch_size + 1,
    initial_epoch    = 0
)